In [1]:
%matplotlib inline
import pennylane as qml
from pennylane import numpy as np
from dimod import BQM, ExactSolver
from neal import SimulatedAnnealingSampler
import pandas as pd
from IPython.display import display, Math

![problem graph](./assets/graph.png)

In [2]:
idx = pd.Index(['A', 'B', 'C', 'D', 'E'])
df = pd.DataFrame([
    np.array([0, 5.8, 5.8, 7.6, 8.8], dtype=np.float16),
    np.array([5.8, 0, 2.4, 4.3, 4.3], dtype=np.float16),
    np.array([5.8, 2.4, 0, 3.5, 4.7], dtype=np.float16),
    np.array([7.6, 4.3, 3.5, 0, 1.4], dtype=np.float16),
    np.array([8.8, 4.3, 4.7, 1.4, 0], dtype=np.float16),
    ], 
    columns=['A', 'B', 'C', 'D', 'E'], 
    index=idx)
df_cap = df.style.set_caption('Points Distances')
df_cap

,A,B,C,D,E
A,0.0,5.8,5.8,7.6,8.8
B,5.8,0.0,2.4,4.3,4.3
C,5.8,2.4,0.0,3.5,4.7
D,7.6,4.3,3.5,0.0,1.4
E,8.8,4.3,4.7,1.4,0.0


---

In [3]:
# testing using only the first step

P = df['A'].sum()

quadratic = {}

for i,v in enumerate(df['A'][1:]):
    print(v)
    quadratic[('x0', f'x{i+1}')] = v
    
qubo = BQM(quadratic, "BINARY")
qubo.add_linear_equality_constraint([(f'x{i}', 1) for i in range(5)], P, -1)
#qubo.to_ising()
print(qubo)

5.8
5.8
7.6
8.8
BinaryQuadraticModel({'x0': -28.0, 'x1': -28.0, 'x2': -28.0, 'x3': -28.0, 'x4': -28.0}, {('x1', 'x0'): 61.80078125, ('x2', 'x0'): 61.80078125, ('x2', 'x1'): 56.0, ('x3', 'x0'): 63.6015625, ('x3', 'x1'): 56.0, ('x3', 'x2'): 56.0, ('x4', 'x0'): 64.796875, ('x4', 'x1'): 56.0, ('x4', 'x2'): 56.0, ('x4', 'x3'): 56.0}, 28.0, 'BINARY')


In [4]:
sampler = ExactSolver()
sampleset = sampler.sample(qubo)
print(sampleset)

   x0 x1 x2 x3 x4     energy num_oc.
1   1  0  0  0  0        0.0       1
3   0  1  0  0  0        0.0       1
7   0  0  1  0  0        0.0       1
15  0  0  0  1  0        0.0       1
31  0  0  0  0  1        0.0       1
0   0  0  0  0  0       28.0       1
4   0  1  1  0  0       28.0       1
8   0  0  1  1  0       28.0       1
12  0  1  0  1  0       28.0       1
16  0  0  0  1  1       28.0       1
24  0  0  1  0  1       28.0       1
28  0  1  0  0  1       28.0       1
2   1  1  0  0  0  33.800781       1
6   1  0  1  0  0  33.800781       1
14  1  0  0  1  0  35.601562       1
30  1  0  0  0  1  36.796875       1
11  0  1  1  1  0      112.0       1
19  0  1  0  1  1      112.0       1
23  0  0  1  1  1      112.0       1
27  0  1  1  0  1      112.0       1
5   1  1  1  0  0 123.601562       1
9   1  0  1  1  0 125.402344       1
13  1  1  0  1  0 125.402344       1
25  1  0  1  0  1 126.597656       1
29  1  1  0  0  1 126.597656       1
17  1  0  0  1  1 128.398438       1
2

In [5]:
result = sampleset.to_pandas_dataframe()
result = result.drop(['num_occurrences'], axis=1)
result.head()

,x0,x1,x2,x3,x4,energy
0,0,0,0,0,0,28.000000
1,1,0,0,0,0,0.000000
2,1,1,0,0,0,33.800781
3,0,1,0,0,0,0.000000
4,0,1,1,0,0,28.000000


In [6]:
min_energy = result['energy'].min()
print('min energy ExactSolver(): ', min_energy)

min energy ExactSolver():  0.0


In [7]:
variables = sampleset.variables.to_serializable()
bit_strings = result.where(result['energy'] == min_energy).dropna()[variables].astype(int).astype(str).agg(''.join, axis=0).tolist()
bit_strings

['10000', '01000', '00100', '00010', '00001']

---

In [8]:
letters = ('A', 'B', 'C', 'D', 'E')

variables = ''
using_letters = ''

for i in range(5):
    for j in range(5):
        if(i == j):
            continue

        using_letters += f'{i}{letters[j]}'
        variables += f'x{i}x{j} '

        if(not (i == 4 and j == 3)):
            variables += '+'
            using_letters += '+'

display(Math(variables))
display(Math(using_letters))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [28]:
letters = ('A', 'B', 'C', 'D', 'E')

variables = ''
using_letters = ''


#format: timestep - last step - actual step
for timestep in range(5):
    if(timestep == 0):
        for i,letter in enumerate(letters[1:]):
            using_letters += f'T{timestep}LAN{letter} + '
            variables += f'T{timestep}L0N{i+1} + '

    elif (timestep == 1):
        for i,prev_letter in enumerate(letters[1:]):
            for j, next_letter in enumerate(letters):
                if(prev_letter == next_letter):
                    continue
            
                using_letters += f'T{timestep}L{prev_letter}N{next_letter} + '
                variables += f'T{timestep}L{i+1}N{j} + '
    else:
        for i, prev_letter in enumerate(letters):
            for j, next_letter in enumerate(letters):
                if(prev_letter == next_letter):
                    continue
            
                using_letters += f'T{timestep}L{prev_letter}N{next_letter}'
                variables += f'T{timestep}L{i}N{j}'

                if(not(i == len(letters)-1 and j == len(letters)-2 and timestep == 4)):
                    using_letters += " +"
                    variables += " +"
        

display(Math(variables))
display(Math(using_letters))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

---

In [3]:
n_steps = df.shape[0]
total_points = df.shape[0]
P = 100

linear = {}
constraints = [set() for _ in range(total_points-1)]

first_pos_data = df.iloc[0].tolist()


for timestep in range(1, n_steps):
    if (timestep == 1):
        for previous_point in range(1,total_points):
            prev_data = df.iloc[previous_point].tolist()
            
            for next_point in range(total_points):
                if(previous_point == next_point):
                    continue

                control_variable = f'p_1_{previous_point}'
                linear[control_variable] = prev_data[next_point]
                constraints[timestep-1].add(control_variable)
                
    else:
        for previous_point in range(total_points):
            prev_data = df.iloc[previous_point].tolist()
            
            for next_point in range(total_points):
                if(previous_point == next_point):
                    continue
                    
                control_variable = f'p_{timestep}_{previous_point}'
                linear[control_variable] = prev_data[next_point]
                constraints[timestep-1].add(control_variable)

print(constraints)
qubo = BQM(vartype="BINARY")
qubo.add_linear_from(linear)
for constraint in constraints:
    qubo.add_linear_equality_constraint([(i, 1) for i in constraint], P, -1)

for i in range(total_points):
    constraint_same_point = []
    
    for constraint in constraints:
        for c in constraint:
            if(int(c[-1]) == i):
                constraint_same_point.append((c,1))
                break
    qubo.add_linear_equality_constraint(constraint_same_point, P, -1)


    
print(qubo)

[{'p_1_4', 'p_1_3', 'p_1_1', 'p_1_2'}, {'p_2_3', 'p_2_0', 'p_2_2', 'p_2_4', 'p_2_1'}, {'p_3_2', 'p_3_4', 'p_3_3', 'p_3_0', 'p_3_1'}, {'p_4_1', 'p_4_0', 'p_4_4', 'p_4_3', 'p_4_2'}]
BinaryQuadraticModel({'p_1_1': -195.69921875, 'p_1_2': -195.30078125, 'p_1_3': -198.599609375, 'p_1_4': -198.599609375, 'p_2_0': -191.203125, 'p_2_1': -195.69921875, 'p_2_2': -195.30078125, 'p_2_3': -198.599609375, 'p_2_4': -198.599609375, 'p_3_0': -191.203125, 'p_3_1': -195.69921875, 'p_3_2': -195.30078125, 'p_3_3': -198.599609375, 'p_3_4': -198.599609375, 'p_4_0': -191.203125, 'p_4_1': -195.69921875, 'p_4_2': -195.30078125, 'p_4_3': -198.599609375, 'p_4_4': -198.599609375}, {('p_1_2', 'p_1_1'): 200.0, ('p_1_3', 'p_1_1'): 200.0, ('p_1_3', 'p_1_2'): 200.0, ('p_1_4', 'p_1_1'): 200.0, ('p_1_4', 'p_1_2'): 200.0, ('p_1_4', 'p_1_3'): 200.0, ('p_2_1', 'p_1_1'): 200.0, ('p_2_1', 'p_2_0'): 200.0, ('p_2_2', 'p_1_2'): 200.0, ('p_2_2', 'p_2_0'): 200.0, ('p_2_2', 'p_2_1'): 200.0, ('p_2_3', 'p_1_3'): 200.0, ('p_2_3', 'p_2

In [124]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(qubo, num_reads=1000)
print(sampleset)

    p_1_1 p_1_2 p_1_3 p_1_4 p_2_0 p_2_1 p_2_2 ... p_4_4     energy num_oc.
2       1     0     0     0     0     0     0 ...     0 111.800781       1
6       0     0     0     1     0     1     0 ...     0 111.800781       1
9       0     0     1     0     0     1     0 ...     0 111.800781       1
15      0     0     1     0     0     0     1 ...     0 111.800781       1
26      0     0     1     0     0     0     0 ...     0 111.800781       1
31      0     0     0     1     0     0     0 ...     0 111.800781       1
39      1     0     0     0     0     0     1 ...     0 111.800781       1
41      1     0     0     0     0     0     0 ...     0 111.800781       1
49      0     0     1     0     0     1     0 ...     1 111.800781       1
57      0     0     1     0     0     0     0 ...     0 111.800781       1
59      0     0     1     0     0     1     0 ...     0 111.800781       1
75      1     0     0     0     0     0     1 ...     1 111.800781       1
76      1     0     0    

In [125]:
result = sampleset.to_pandas_dataframe()
print('Saving CSV file...')
result.to_csv("first-test-result.csv")

Saving CSV file...


In [126]:
min_energy = result['energy'].min()
print('min energy SimulatedAnnealingSampler(): ', min_energy)

min energy SimulatedAnnealingSampler():  111.80078125


In [127]:
lowest_energies = sampleset.lowest().to_pandas_dataframe()

lowest_energies

,p_1_1,p_1_2,p_1_3,p_1_4,p_2_0,p_2_1,p_2_2,p_2_3,p_2_4,p_3_0,p_3_1,p_3_2,p_3_3,p_3_4,p_4_0,p_4_1,p_4_2,p_4_3,p_4_4,energy,num_occurrences
0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,111.800781,1
1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,111.800781,1
2,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,111.800781,1
3,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,111.800781,1
4,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,111.800781,1
5,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,111.800781,1
6,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,111.800781,1
7,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,111.800781,1
8,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,111.800781,1
9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,111.800781,1
